In [1]:
import pandas as pd

from sqlalchemy import create_engine
import sqlite3
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import func
import json

In [2]:
#Reading the json file
jsondata = pd.read_json("data/samples.json")
jsondata

,names,metadata,samples
0,38,"{'Node_id': 38, 'Gateway_ID': 21545, 'Model': ...","{'Node_id': '38', 'otu_ids': ['5/05/2016', '6/..."
1,39,"{'Node_id': 39, 'Gateway_ID': 21545, 'Model': ...","{'Node_id': '39', 'otu_ids': ['5/05/2016', '6/..."
2,40,"{'Node_id': 40, 'Gateway_ID': 21545, 'Model': ...","{'Node_id': '40', 'otu_ids': ['5/05/2016', '6/..."
3,41,"{'Node_id': 41, 'Gateway_ID': 21545, 'Model': ...","{'Node_id': '41', 'otu_ids': ['5/05/2016', '6/..."
4,42,"{'Node_id': 42, 'Gateway_ID': 21545, 'Model': ...","{'Node_id': '42', 'otu_ids': ['5/05/2016', '6/..."


In [3]:
#Checking the datatype of each column
jsondata.dtypes

names        int64
metadata    object
samples     object
dtype: object

In [4]:
#Converting the columns type as mentioned
#If not converted shows error while fetchong the data in the table
jsondata['names']= jsondata['names'].astype('str')
jsondata['metadata']= jsondata['metadata'].astype('str')
jsondata['samples']= jsondata['samples'].astype('str')
#jsondata['metadata'] = jsondata['metadata'].apply(json.dumps).astype('string')

In [5]:
jsondata.dtypes

names       object
metadata    object
samples     object
dtype: object

In [6]:
#Creating the database "peizodata.sqlite"
engine = create_engine("sqlite:///peizodata.sqlite", echo=False)
conn = engine.connect()

In [7]:

#Drop table if it exists
engine.execute('DROP TABLE IF EXISTS peizometer;')
#Create table and specify primary key
engine.execute('CREATE TABLE "peizometer" ('
               'names TEXT,'
               'metadata TEXT, '
                'samples TEXT,'
               'PRIMARY KEY (names));')

In [8]:
#jsondata.to_sql('peizometer',con=engine,if_exists='append',index=False)

#Converting the "jsondata dataframe" to sqlite file and replacing the data in the table named "peizometer"
jsondata.to_sql(name ='peizometer',con=engine,if_exists='replace',index=False)

In [9]:
#Checking if the data reads from the table
engine.execute("select * from peizometer").fetchall()

[('38', "{'Node_id': 38, 'Gateway_ID': 21545, 'Model': 'LS-G6-VW-5-FCC', 'Piezometer_Channel': 5, 'Piezometer_ID': 'VW38', 'Installation_Date': '5/05/2018', ' ... (105 characters truncated) ... d': 15.02020495, 'Tip Elevation': 605, 'Maxpressure': -10.97, 'Minimumpressure': -22.03, 'Date_last_reading': '20/03/2021', 'Latest_pressure': -21.8}", "{'Node_id': '38', 'otu_ids': ['5/05/2016', '6/05/2016', '7/05/2016', '8/05/2016', '9/05/2016', '10/05/2016', '11/05/2016', '12/05/2016', '13/05/2016' ... (6889 characters truncated) ... -21.75, -21.82, -21.9, -21.87, -21.72, -21.68, -21.7, -21.82, -21.79, -21.92, -22.03, -21.79, -21.81, -21.93, -21.93, -21.79, -21.81, -21.85, -21.8]}"),
 ('39', "{'Node_id': 39, 'Gateway_ID': 21545, 'Model': 'LS-G6-VW-5-FCC', 'Piezometer_Channel': 4, 'Piezometer_ID': 'VW39', 'Installation_Date': '5/05/2018', ' ... (103 characters truncated) ... tud': 15.02020495, 'Tip Elevation': 603, 'Maxpressure': 23.83, 'Minimumpressure': 10.31, 'Date_last_reading': '20/03/202

In [10]:
inspector = inspect(engine)

# Collect the names of tables within the database
inspector.get_table_names()

['node', 'peizometer', 'sample']

In [11]:
#To get the names of columns in the table
columns = inspector.get_columns('peizometer')
for column in columns:
    print(column["name"], column["type"])

names TEXT
metadata TEXT
samples TEXT


In [12]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)

#This reflect the keys after we assign the primary key in the sqlite database
Base.classes.keys()

['node', 'sample']